<a href="https://colab.research.google.com/github/malihe1988/AIF360/blob/master/test_aif360_6_adult_predjuce_remover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aif360
!pip install 'aif360[LawSchoolGPA]'
!pip install 'aif360[Reductions]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 9.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 13.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 9.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_Adult_df.csv to final_Adult_df.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['final_Adult_df.csv']))

In [ ]:
df.head()

,age,fnlwgt,education.num,race,sex,capital.gain,capital.loss,hours.per.week,income,workclass_Local-gov,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,90,77053,9,1,0,0,4356,40,0,0,...,0,0,0,0,0,0,0,1,0,0
1,82,132870,9,1,0,0,4356,18,0,0,...,0,0,0,0,0,0,0,1,0,0
2,66,186061,10,0,0,0,4356,40,0,0,...,0,0,0,0,0,0,0,1,0,0
3,54,140359,4,1,0,0,3900,40,0,0,...,0,0,0,0,0,0,0,1,0,0
4,41,264663,10,1,0,0,3900,40,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#Here is the code that allows us to leverage the AIF360 library.


from sklearn.model_selection import train_test_split
from aif360.datasets import BinaryLabelDataset
df_train, df_test = train_test_split(df, test_size=0.2)
attributes_params = dict(
    protected_attribute_names=["sex"],
    label_names=["income"]
)
dt_train = BinaryLabelDataset(df=df_train, **attributes_params)
dt_test = BinaryLabelDataset(df=df_test, **attributes_params)

In [ ]:
#Strategy n°1 : do nothing
#This is just what people do when they are not aware of fairness. Let us train a logistic regression on 𝑋𝑠𝑘𝑖𝑙𝑙, 𝑋𝑠𝑎𝑙𝑎𝑟𝑦, 𝑋𝑠𝑒𝑥 and see what happens.

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
preprocessing = ColumnTransformer(
    [("scaler", RobustScaler(), [0, 1])],
    remainder="passthrough"
)
lr = Pipeline([
    ("preprocessing", preprocessing),
    ("lr", LogisticRegression(C=1e9))
])
lr.fit(
    dt_train.features,
    dt_train.labels.ravel()
);
y_pred = lr.predict(dt_test.features)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Strategy n°5 : in-processing with constraints
#This strategy is nothing but a logistic regression with an additional penalty encoding the prejudice made by the algorithm.
from aif360.algorithms.inprocessing import PrejudiceRemover
pr = PrejudiceRemover(eta=100.0)
dt_train.features = preprocessing.fit_transform(dt_train.features)
dt_test.features = preprocessing.transform(dt_test.features)
pr.fit(dt_train);
y_pred = pr.predict(dt_test).scores >= 0.5